In [1]:
import numpy as np
from numpy import pi
from numpy import exp
from numpy import sin
from numpy import deg2rad
from numpy import rad2deg
from numpy.linalg import eig

In [1]:
def STD_MUSIC(SNR = 20, snapshots = 200, sensorNum = 8):
#----------Consider a ULA, where the array sapcing is a half wavelength of the signal.--------#
    c = 3e8
    f = 2.4e9                       # frequency is 2.4GHz
    wavelength = c / f              # lambda
    spacing = wavelength / 2        # ULA's spacing

#---------Sample: sample frequency is fs = 3f-------------------------------------------#
    fs = 3 * f                      
    Ts = 1 / fs                             # Sample period
    Ns = Ts * np.arange(snapshots)          # Sample spacing

#----------Consider noises are generated from a zero mean Gaussian distribution.--------------#
    sigma_N = 0.1
    noiseCovMat = np.diag(sigma_N * np.ones(sensorNum))    

    noiseAmp = np.random.multivariate_normal(np.zeros(sensorNum), noiseCovMat, snapshots)
    noisePhase = np.mat([exp(-1j*2*pi*f*Ns + np.random.rand())])
    noiseMat = np.multiply(noiseAmp, noisePhase.T)                    # Each row is A sample 

#----------Consider three uncorrlated sources at -10 degree, 0 degree and 10 degree,-------#
#----------Each source is generated from a zerom mean Gaussian distribution.------------------#    
    theta_S = np.array([-10, 0, 10])
    sourceNum = len(theta_S)
    sigma_S = sigma_N * 10**(SNR/20)
    signalCovMat = np.mat([[sigma_S, 0.99*sigma_S, 0], [0.99*sigma_S, sigma_S, 0], [0, 0, sigma_S]])
    signalAmp = np.random.multivariate_normal(np.zeros(sourceNum), signalCovMat, snapshots)
    signalPhase = np.mat([exp(-1j*2*pi*f*Ns + np.random.rand())])
    signalMat = np.multiply(signalAmp, signalPhase.T)                # Each row is A sample 

    spacingK = spacing * np.arange(sensorNum)
    manifoldMat = np.zeros((sensorNum, sourceNum), dtype=complex)
    for col in range(manifoldMat.shape[1]):
        manifoldMat[:, col] = np.mat([exp(-1j*2*pi*f*((spacingK*sin(deg2rad(theta_S[col])))/c))], dtype=complex)

    sensorOut = np.dot(manifoldMat, signalMat.T) + noiseMat.T
    covMat_Hat = np.dot(sensorOut, sensorOut.H) / snapshots
    eigenVals, eigenVects = eig(covMat_Hat)
    eigenValsIdx = np.argsort(-eigenVals)
    noisySubspace = eigenVects[:, eigenValsIdx[np.arange(sourceNum, sensorNum)]]

    
    theta = np.arange(-90, 90, 0.1)
    P_MUSIC = np.zeros(len(theta), dtype=complex)
    for itr in range(len(theta)):
        steeringVec = np.mat(exp(-1j*2*pi*f*(spacingK*sin(deg2rad(theta[itr]))/c))).T
        P_MUSIC[itr] = 1/(steeringVec.H*noisySubspace*noisySubspace.H*steeringVec)
    
    return theta, P_MUSIC

In [1]:
def Spatial_Smoothing_MUSIC(SNR = 20, n = 200, M = 8):
#----------SNR: signal noise radio, n: the number of snapshots, M: the number of array.-------#
#----------Consider a ULA, where the array sapcing is a half wavelength of the signal.--------#
    c = 3e8
    f = 2.4e9
    lamb = c/f
    spacing = lamb/2

#---------Sample: the number of snapshots = n = 200-------------------------------------------#
    fs = 3*f
    Ts = 1/fs
    Ns = Ts*np.arange(n)

#     print('SNR:', SNR)
#     print('Snapshots:', n)
#     print('The number of sensors:', M)

#----------Consider noises are generated from a zero mean Gaussian distribution.--------------#
    sigma_N = 0.1
    R_N = np.diag(sigma_N * np.ones(M))    
#     print('Covariance matrix of noise:\n')
#     print(R_N)

    N = np.random.multivariate_normal(np.zeros(M), R_N, n)
    phase = np.mat([np.exp(-1j*2*pi*f*Ns + np.random.rand())])
    N = np.multiply(N, phase.T)                    # N is noisy matrix generated from a Gaussian

#----------Consider three sources at -10 degree, 0 degree and 10 degree.----------------------#
#----------Sources at -10 degree and 0 degree is coherent.------------------------------------#
#----------Each source is generated from a zerom mean Gaussian distribution.------------------#
    theta_S = np.array([-10, 0, 10])
    sigma_S = sigma_N * np.power(10, (SNR/20))
    R_S = np.mat([[sigma_S, 0.99*sigma_S, 0], [0.99*sigma_S, sigma_S, 0], [0, 0, sigma_S]])

    S = np.random.multivariate_normal(np.zeros(len(theta_S)), R_S, n)
    S = np.multiply(S, phase.T)                    # S is signal matrix generated from a Gaussian 

    spacingK = spacing * np.arange(M)
    A = np.zeros((M, theta_S.size), dtype=complex)
    for col in range(A.shape[1]):
        A[:, col] = np.mat([np.exp(-1j*2*pi*f*(spacingK*np.sin(np.deg2rad(theta_S[col]))) / c)], dtype=complex)

#     print('Each row is a sample')
#     print('Signal matrix S size:', S.shape)
#     print('Noise matrix N Size:', N.shape)
#     print('Manifold matrix A size:', A.shape)

    X = np.dot(A, S.T) + N.T

    R_h = np.dot(X, X.H) / n                        # R hat: covariance matrix of samples

    m = round((M + (len(theta_S)/2) + 1)/2)                    # m S.T. D <= m <= M - D/2 + 1
    p = M + 1 - m
    R_fk = np.mat(np.zeros((m, m)))
    R_bk = np.mat(np.zeros((m, m)))

    for k in range(1, p + 1):
        Z_k = np.mat(np.hstack((np.zeros((m, k - 1)), np.eye(m), np.zeros((m, p - k)))))
        R_fk = R_fk + Z_k*R_h*Z_k.H

        Q_k = np.mat(np.hstack((np.zeros((m, k - 1)), np.fliplr(np.eye(m)), np.zeros((m, p - k)))))
        R_bk = R_bk + Q_k*np.conj(R_h)*Q_k.H

    R_f = R_fk/p
    R_b = R_bk/p
    R_fb = (R_f + R_b)/2

    D, V = eig(R_fb)
    D_index = np.argsort(D)
    numOfSources = len(theta_S)
    V = V[:, D_index]
    U_n = V[:, np.arange(m - numOfSources)]

    theta = np.arange(-90, 90, 0.1)
    P_mu = np.zeros(len(theta), dtype=complex)
    for itr in range(len(theta)):
        steeringVec = np.mat(exp(-1j*2*pi*f*(spacing*np.arange(m)*sin(deg2rad(theta[itr]))/c))).T
        P_mu[itr] = 1/(steeringVec.H*U_n*U_n.H*steeringVec)
    
    return theta, P_mu

In [2]:
def SS_MUSIC(SNR = 20, snapshots = 200, sensorNum = 8):
#----------Consider a ULA, where the array sapcing is a half wavelength of the signal.--------#
    c = 3e8
    f = 2.4e9                       # frequency is 2.4GHz
    wavelength = c / f              # lambda
    spacing = wavelength / 2        # ULA's spacing

#---------Sample: sample frequency is fs = 3f-------------------------------------------#
    fs = 3 * f                      
    Ts = 1 / fs                             # Sample period
    Ns = Ts * np.arange(snapshots)          # Sample spacing

#----------Consider noises are generated from a zero mean Gaussian distribution.--------------#
    sigma_N = 0.1
    noiseCovMat = np.diag(sigma_N * np.ones(sensorNum))    

    noiseAmp = np.random.multivariate_normal(np.zeros(sensorNum), noiseCovMat, snapshots)
    noisePhase = np.mat([exp(-1j*2*pi*f*Ns + np.random.rand())])
    noiseMat = np.multiply(noiseAmp, noisePhase.T)                    # Each row is A sample 

#----------Consider four uncorrlated sources at -25 degree, 0 degree and 25 degree,-------#
#----------Each source is generated from a zerom mean Gaussian distribution.------------------#    
    theta_S = np.array([-10, 0, 10])
    sourceNum = len(theta_S)
    sigma_S = sigma_N * 10**(SNR/20)
    signalCovMat = np.diag(sigma_S * np.ones(sourceNum))
    signalAmp = np.random.multivariate_normal(np.zeros(sourceNum), signalCovMat, snapshots)
    signalPhase = np.mat([exp(-1j*2*pi*f*Ns + np.random.rand())])
    signalMat = np.multiply(signalAmp, signalPhase.T)                # Each row is A sample 

    spacingK = spacing * np.arange(sensorNum)
    manifoldMat = np.zeros((sensorNum, sourceNum), dtype=complex)
    for col in range(manifoldMat.shape[1]):
        manifoldMat[:, col] = np.mat([exp(-1j*2*pi*f*((spacingK*sin(deg2rad(theta_S[col])))/c))], dtype=complex)

    sensorOut = np.dot(manifoldMat, signalMat.T) + noiseMat.T
    covMat_Hat = np.dot(sensorOut, sensorOut.H) / snapshots
    eigenVals, eigenVects = eig(covMat_Hat)
    eigenValsIdx = np.argsort(-eigenVals)
    signalSubspace = eigenVects[:, eigenValsIdx[np.arange(sourceNum)]]
    noisySubspace = eigenVects[:, eigenValsIdx[np.arange(sourceNum, sensorNum)]]

    noisyPower_Hat = (1/(sensorNum - sourceNum))*np.sum(eigenVals[np.arange(sourceNum, sensorNum)])
    R = np.mat(np.diag(eigenVals[eigenValsIdx[np.arange(sourceNum)]]) - noisyPower_Hat*np.eye(sourceNum))
    noiseFreeMatPinv = np.dot(np.dot(signalSubspace, R.I), signalSubspace.H)

    theta = np.arange(-90, 90, 0.1)
    P_SS_MUSIC = np.zeros(len(theta), dtype=complex)
    for itr in range(len(theta)):
        steeringVec = np.mat(exp(-1j*2*pi*f*(spacingK*sin(deg2rad(theta[itr]))/c))).T
        P_SS_MUSIC[itr] = (steeringVec.H*noiseFreeMatPinv*steeringVec) / (steeringVec.H*noisySubspace*noisySubspace.H*steeringVec)

    return theta, P_SS_MUSIC